# Surprize

In [104]:
import sys
import os as os
import numpy as np
try:
    import cPickle as pickle
except:
    import pickle as pkl

import scipy as scipy
import scipy.io as spio
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import signal
from scipy.signal import find_peaks
from scipy.ndimage import gaussian_filter1d
from scipy.io.matlab import mat_struct
import pickle
import random
random.seed(666)

from sklearn.mixture import GaussianMixture
from matplotlib.patches import Ellipse

def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    data = spio.loadmat(filename, struct_as_record=True, squeeze_me=True)
    return _check_keys(data)

def _check_keys(dict):
    '''
    checks if entries in dictionary are mat-objects. If yes
    todict is called to change them to nested dictionaries
    '''
    for key in dict:
        if isinstance(dict[key], scipy.io.matlab.mat_struct):
            dict[key] = _todict(dict[key])
    return dict        

def _todict(matobj):
    '''
    A recursive function which constructs from matobjects nested dictionaries
    '''
    dict = {}
    for strg in matobj._fieldnames:
        elem = matobj.__dict__[strg]
        if isinstance(elem, scipy.io.matlab.mat_struct):
            dict[strg] = _todict(elem)
        else:
            dict[strg] = elem
    return dict

In [105]:
meta = {}
meta[45] = {'mouse':'3C280', 'ks':'Kilosort_2024-04-12_162032', 'del_units':[594], 'vis_mice':np.array(['nan', 'beta', 'alpha'])}
meta[46] = {'mouse':'3C280', 'ks':'Kilosort_2024-04-12_180855', 'del_units':None, 'vis_mice':np.array(['nan', 'alpha', 'beta'])}
meta[11] = {'mouse':'3C290', 'ks':'Kilosort_2024-05-06_154258', 'del_units':[847, 835], 'vis_mice':np.array(['nan', 'beta', 'alpha'])}
meta[13] = {'mouse':'3C290', 'ks':'Kilosort_2024-05-14_120055', 'del_units':None, 'vis_mice':np.array(['nan', 'beta', 'alpha'])}
meta[14] = {'mouse':'3C290', 'ks':'Kilosort_2024-05-14_122629', 'del_units':None, 'vis_mice':np.array(['nan', 'beta', 'alpha'])}
meta[19] = {'mouse':'3C290', 'ks':'Kilosort_2024-05-14_140410', 'del_units':None, 'vis_mice':np.array(['nan', 'beta', 'alpha'])}
meta[20] = {'mouse':'3C290', 'ks':'Kilosort_2024-05-15_110539', 'del_units':[33], 'vis_mice':np.array(['nan', 'beta', 'alpha'])}

In [106]:
Session = 20
mouse = meta[Session]['mouse']
vis_mice = meta[Session]['vis_mice']
vis_mice = list(vis_mice)
vis_mice.append('blank')
vis_mice = vis_mice[1:]

In [107]:
matfile =  f'L:\\everyone\\sharedDATA\\ProcessedDATA\\{mouse}\\{mouse}_S{Session}\\PSTH.mat'
surprize = loadmat(matfile)

In [108]:
alpha = 0.01

In [109]:
active_blocks = [1,2,3]
PSTH_dic = {}
for block in active_blocks:
    psth_block = {}
    psth_block['Pi_poi'] = surprize['PSTH'][block][6][:,0]
    psth_block['Pd_poi'] = surprize['PSTH'][block][6][:,1]
    psth_block['sum_i'] = np.sum(surprize['PSTH'][block][6][:,0]<alpha)
    psth_block['sum_d'] = np.sum(surprize['PSTH'][block][6][:,1]<alpha)
    psth_block['idx_i'] = np.where(surprize['PSTH'][block][6][:,0]<alpha)[0]
    psth_block['idx_d'] = np.where(surprize['PSTH'][block][6][:,1]<alpha)[0]
    PSTH_dic[block] = psth_block

In [110]:
i_list = []
d_list = []
for block in active_blocks:
    i_list.append(PSTH_dic[block]['idx_i'])
    d_list.append(PSTH_dic[block]['idx_d'])

In [111]:
i_list,

([array([11], dtype=int64),
  array([11], dtype=int64),
  array([ 1,  2,  6, 13], dtype=int64)],)

In [112]:
d_list


[array([ 3,  4,  8,  9, 12, 14, 15], dtype=int64),
 array([ 3,  8, 14], dtype=int64),
 array([3], dtype=int64)]

In [113]:
vis_mice

['beta', 'alpha', 'blank']

In [114]:
i_d_results = {}

In [115]:
# Corresponding names (example order)
names = vis_mice

# Create a dictionary to map each name to its corresponding set
sets = {
    names[0]: set(i_list[0]),
    names[1]: set(i_list[1]),
    names[2]: set(i_list[2])
}

# Perform set operations
results_i = {
    f'only_in_{names[0]}': len(sets[names[0]] - sets[names[1]] - sets[names[2]]),
    f'only_in_{names[1]}': len(sets[names[1]] - sets[names[0]] - sets[names[2]]),
    f'only_in_{names[2]}': len(sets[names[2]] - sets[names[0]] - sets[names[1]]),
    f'in_{names[0]}_and_{names[1]}': len((sets[names[0]] & sets[names[1]]) - sets[names[2]]),
    f'in_{names[1]}_and_{names[2]}': len((sets[names[1]] & sets[names[2]]) - sets[names[0]]),
    f'in_{names[0]}_and_{names[2]}': len((sets[names[0]] & sets[names[2]]) - sets[names[1]]),
    f'in_all_three': len(sets[names[0]] & sets[names[1]] & sets[names[2]])
}
i_d_results['i'] = results_i
# Print the results
for key, value in results_i.items():
    print(f"{key}: {value}")

only_in_beta: 0
only_in_alpha: 0
only_in_blank: 4
in_beta_and_alpha: 1
in_alpha_and_blank: 0
in_beta_and_blank: 0
in_all_three: 0


In [116]:
# Corresponding names (example order)
names = vis_mice

# Create a dictionary to map each name to its corresponding set
sets = {
    names[0]: set(d_list[0]),
    names[1]: set(d_list[1]),
    names[2]: set(d_list[2])
}

# Perform set operations
results_d = {
    f'only_in_{names[0]}': len(sets[names[0]] - sets[names[1]] - sets[names[2]]),
    f'only_in_{names[1]}': len(sets[names[1]] - sets[names[0]] - sets[names[2]]),
    f'only_in_{names[2]}': len(sets[names[2]] - sets[names[0]] - sets[names[1]]),
    f'in_{names[0]}_and_{names[1]}': len((sets[names[0]] & sets[names[1]]) - sets[names[2]]),
    f'in_{names[1]}_and_{names[2]}': len((sets[names[1]] & sets[names[2]]) - sets[names[0]]),
    f'in_{names[0]}_and_{names[2]}': len((sets[names[0]] & sets[names[2]]) - sets[names[1]]),
    f'in_all_three': len(sets[names[0]] & sets[names[1]] & sets[names[2]])
}
i_d_results['d'] = results_d
# Print the results
for key, value in results_d.items():
    print(f"{key}: {value}")

only_in_beta: 4
only_in_alpha: 0
only_in_blank: 0
in_beta_and_alpha: 2
in_alpha_and_blank: 0
in_beta_and_blank: 0
in_all_three: 1


In [117]:
try:
    os.makedirs(f'C:\\Users\\ebukina\\Desktop\\eva\\results\\surprize')
except:
    pass

In [118]:
save_path = f'C:\\Users\\ebukina\\Desktop\\eva\\results\\surprize\\{mouse}_S{Session}_surprize.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(i_d_results, f)

In [119]:
save_path = f'C:\\Users\\ebukina\\Desktop\\eva\\results\\surprize\\{mouse}_S{Session}_PSTH_dic.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(PSTH_dic, f)

In [120]:
# only_in_alpha: 0
# only_in_beta: 3
# only_in_blank: 1
# in_alpha_and_beta: 4
# in_beta_and_blank: 3
# in_alpha_and_blank: 4
# in_all_three: 5


# only_in_beta: 0
# only_in_alpha: 3
# only_in_blank: 1
# in_beta_and_alpha: 4
# in_alpha_and_blank: 3
# in_beta_and_blank: 4
# in_all_three: 5